# Setup

This is the source file for classes and functions used throughout the project.

In [5]:
try:
    %load_ext lab_black
except ModuleNotFoundError:
    print("Couldn't load Black autoformatter.")

In [15]:
import numpy as np
from IPython.core.display import display
from numpy import random
import pandas as pd
import os, sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm

In [2]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
PROJECT_SAVE_DIR = "figs"

if not (os.path.isdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)):
    print('Figure directory did not exist, creating now.')
    os.mkdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)
else:
    print('Figure directory exists.')

Figure directory exists.


In [3]:
# Read in target (ENM) model feature data
X_enm = pd.read_csv("./data/ENM-preprocessed-feats.csv", 
                    sep='\t', header='infer', index_col=0)

# Read in source (organics) model feature data
X_source = pd.read_csv("./data/organics-preprocessed-feats.csv", 
                       sep='\t', header='infer', index_col=0)

# Read in ENM labels (maximum_weight_fraction)
y_enm = pd.read_csv("./data/ENM-clean.csv", 
                    sep=',', header='infer', usecols=[4])

# Read in organics labels (maximum_weight_fraction)
y_source = pd.read_csv("./data/organics-preprocessed-WF.csv", 
                       sep='\t', header='infer', index_col=0)
y_source.index = X_source.index

# Utility classes, functions

In [26]:
def savepdf(fig, name):
    """Save figures as .pdf files"""

    # Adjust matplotlib settings so text is editable in PDFs
    plt.rcParams["font.family"] = "sans-serif"
    plt.rcParams["font.sans-serif"] = "Helvetica"
    plt.rcParams["pdf.fonttype"] = 42
    plt.rcParams["ps.fonttype"] = 42
    # Save PDF in figure directory
    fig.savefig(
        PROJECT_ROOT_DIR + "/" + PROJECT_SAVE_DIR + "/" + name + ".pdf",
        bbox_inches="tight",
        transparent=True,
    )

In [27]:
class HiddenPrints:
    """
    Option to suppress print output.
    
    Source:
    https://stackoverflow.com/questions/8391411/suppress-calls-to-print-python
    """

    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [28]:
import functools


class CompletionNotifier:
    """Nestable function-completion notification decorator
    
    I.e., no notification sound on inner functions.
    
    Source:
    https://gist.github.com/jdpage/26376472ac18a7057e381f2259c7b988
    """

    def __init__(self, notify_fun):
        self.notify_fun = notify_fun
        self.isinner = False

    def __call__(self, f):
        # return f # to disable
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            isinner = self.isinner
            self.isinner = True
            try:
                return f(*args, **kwargs)
            finally:
                self.isinner = isinner
                if not self.isinner:
                    self.notify_fun(f)

        return wrapper


# To enable notification sound to play
def ping_notify(*args):
    from IPython.display import Audio

    sound_file = "./data/ping.wav"
    display(Audio(url=sound_file, autoplay=True))


notify_on_complete = CompletionNotifier(ping_notify)

# Pre-processing functions

In [29]:
def bins(row):
    """Assign weight fractions (continuous) to bins (int)
    
    Class ranges are different from those used by Isaacs et al. 2016.
    """

    if row["maximum_weight_fraction"] <= 0.01:
        val = 0  # low
    elif row["maximum_weight_fraction"] > 0.10:
        val = 2  # high
    else:
        val = 1  # medium
    return val

In [30]:
# def bins(row):
#    """Assign weight fractions (continuous) to bins (int)
#
#    Class ranges are different from those used by Isaacs et al. 2016.
#    """
#
#    if row['maximum_weight_fraction'] < 0.0001:
#        val = 0 # xlow
#    elif row['maximum_weight_fraction'] >= 0.2:
#        val = 3 # high
#    elif row['maximum_weight_fraction'] >= 0.01:
#        val = 2 # medium
#    else:
#        val = 1 # low
#    return val

In [31]:
# def bins(row):
#    """Assign weight fractions (continuous) to bins (int)
#
#    Class ranges are different from those used by Isaacs et al. 2016.
#    """
#
#    if row['maximum_weight_fraction'] <= 0.0001:
#        val = 0 # low
#    elif row['maximum_weight_fraction'] > 0.01:
#        val = 2 # high
#    else:
#        val = 1 # medium
#    return val

In [32]:
bin_enm = np.asarray(y_enm.apply(bins, axis=1))
bin_source = np.asarray(y_source.apply(bins, axis=1))

In [33]:
def bar_graph_bins(label_data, data_composition):
    """Bar graph of weight fraction bins
    
    Create a bar graph of weight fraction bins and print the 
    count and frequency for each.
    
    Arguments
    ---------
    label_data : int array of shape [n,]
        Dataframe containing binned wf data
    data_composition : string
        Describes the chemical composition of label_data 
        for use in the plot title; e.g., `ENM`, `Organics`   
    """

    import matplotlib.pyplot as plt

    # Find the count, frequency of WF bins
    unique, counts = np.unique(label_data, return_counts=True)
    wf_distrib = dict(zip(unique, counts))
    freq = []
    for i in counts:
        percent = (i / np.sum(counts)).round(2)
        freq.append(percent)
    bin_names = ["low", "medium", "high"]
    if len(unique) == 4:
        bin_names = ["xlow"] + bin_names
    # Plot
    plt.bar(range(len(wf_distrib)), list(wf_distrib.values()), align="center")
    plt.xticks(range(len(wf_distrib)), list(bin_names))
    plt.title("Frequency of %s Weight Fraction Bins" % data_composition)
    plt.show()

    print("Label bin: ", unique)
    print("Count    : ", counts)
    print("Frequency: ", freq)

In [ ]:
# Shapiro-Wilk Test for normality
from scipy.stats import shapiro


def norm_test(data):
    """
    Source:
    https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/
    """
    # Normality test
    stat, p = shapiro(data)
    print("Statistics=%.3f, p=%.3f" % (stat, p))
    # Interpret
    alpha = 0.05
    if p > alpha:
        print("Sample looks Gaussian (fail to reject H0)")
    else:
        print("Sample does not look Gaussian (reject H0)")

In [34]:
def feat_agglom(n_clust, prefix, df_fit, df_trans=None):
    """Apply feature agglomeration to get a list of new column names
    
    If `df_trans` is provided, returns the reduced feature data frame.
    `prefix` is the four character indicator that the column was agglomerated.
    """

    from sklearn.cluster import FeatureAgglomeration

    # ===== Fit feature agglomeration =====
    agg = FeatureAgglomeration(n_clust, affinity="cosine", linkage="average")
    agg.fit(df_fit + 0.0001)  # Add small fraction so features don't disappear

    # ===== Get agglomerated feature labels =====
    # Create array showing order for agglomerated features
    ord_arr = np.column_stack((agg.labels_, df_fit.columns))
    # Agglomerate names of agglomerated features
    len_orig = len(df_fit.columns)
    kids = agg.children_
    for i in np.arange(0, len_orig - n_clust):
        str1, str2 = df_fit.columns[kids[i, 0]], df_fit.columns[kids[i, 1]]
        name = "_".join([prefix, str1[5:], str2[5:]])
        ord_arr[ord_arr == str1] = name
        ord_arr[ord_arr == str2] = name
    # Sort order
    ord_arr = ord_arr[ord_arr[:, 0].argsort()]
    # Remove duplicates
    agg_cols = list(pd.unique(ord_arr[:, 1]))

    # ===== Apply agglomeration =====
    if df_trans is not None:
        df_red = pd.DataFrame(agg.transform(df_trans), columns=agg_cols)
        # Alphabetize features
        agg_cols.sort()
        df_red = df_red[agg_cols]
        return df_red
    else:
        return agg_cols

# Plot functions

In [6]:
# Function for plotting piecharts
def plot_piechart(
    data, feat_subset_prefix, save_fig_name, figsize=[3, 2.5], labels=None
):

    # Aesthetic settings
    font_body = {"fontsize": 7, "fontname": "Helvetica"}
    my_colors = [
        "tab:blue",  # sky blue
        "#E59400",  # orange
        "tab:purple",  # purple-blue
        "#C0504D",  # red
        "tab:olive",
        "#323299",  # dark blue
        "#7a307a",  # violet
        "seagreen",
        "sienna",
        "gold",
    ]

    # Define data and labels for plotting
    feat_subset = [f for f in data.columns if feat_subset_prefix in f]
    if len(feat_subset) == 1:
        pos_labels = np.count_nonzero(data[feat_subset])
        values = [(pos_labels), (len(data[feat_subset]) - pos_labels)]
        if labels is None:
            labels = [feat_subset[0].split("_")[1], "other"]
        else:
            labels = labels

    else:
        values = data[feat_subset].sum(axis=0)
        labels = [f.split("_")[1] for f in feat_subset]

    # Plot
    fig, ax = plt.subplots(figsize=figsize)
    ax.pie(
        x=values,
        autopct="%1.1f%%",
        colors=my_colors,
        labels=labels,
        pctdistance=0.9,
        labeldistance=1.05,
        startangle=90,
        counterclock=False,
        textprops={**font_body},
    )
    ax.axis("equal")
    savepdf(fig, "pie_%s" % save_fig_name.lower().replace(" ", "_").replace(")", ""))
    plt.show()

In [16]:
# Heatmap showing feature correlation (nonparametric)
def correlation_matrix(df, figsize=(6, 6), save_fig_name=None):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    cmap = matplotlib.cm.get_cmap("coolwarm", 24)
    cax = ax.imshow(df.corr("spearman"), cmap=cmap, vmin=-1, vmax=1)
    plt.title("Feature Correlation")
    labels = [f.split("_")[1] for f in df.columns.tolist()]
    ax.set_xticks(range(len(df.columns)))
    ax.set_yticks(range(len(df.columns)))
    ax.set_xticklabels(labels, fontsize=8, rotation=90)
    ax.set_yticklabels(labels, fontsize=8)
    fig.colorbar(cax)

    if np.all(save_fig_name != None):
        savepdf(fig, "feature_correlation_%s" % save_fig_name)

    plt.show()

# Modeling classes, functions

In [36]:
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import multiprocessing


class EstimatorSelectionHelper:
    """
    Set up grid search across multiple estimators, pipelines; automatically 
    performs stratified CV if labels are multiclass.
    
    By David Bastista:
    http://www.davidsbatista.net/blog/2018/02/23/model_optimization/
    """

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError(
                "Some estimators are missing parameters: %s" % missing_params
            )
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=5, n_jobs=1, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(
                model,
                params,
                cv=cv,
                n_jobs=n_jobs,
                verbose=verbose,
                scoring=scoring,
                refit=refit,
                return_train_score=True,
            )
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by="mean_score"):
        def row(key, scores, params):
            d = {
                "estimator": key,
                "min_score": min(scores),
                "max_score": max(scores),
                "mean_score": np.mean(scores),
                "std_score": np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_["params"]
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values(
            ["mean_score", "max_score"], ascending=[False, False]
        )

        columns = ["estimator", "min_score", "mean_score", "max_score", "std_score"]
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [37]:
def plot_feat_impt(
    feat_names, importances, variances=None, save_fig_name=None, combo_impt=False
):
    """Plot bar graph of feature importance
    
    This function uses results from an rfc as input to plot feature 
    importance. Here, the rfc determines importance using gini importance, 
    aka mean decrease impurity. Includes option to combine features into 
    more easily interpretable groups.
    
    References:
    https://stackoverflow.com/questions/15810339/how-are-feature-importances-in-randomforestclassifier-determined
    https://matplotlib.org/examples/api/barchart_demo.html
    https://stackoverflow.com/questions/28931224/adding-value-labels-on-a-matplotlib-bar-chart
    https://stackoverflow.com/questions/14849293/python-find-index-position-in-list-based-of-partial-string
    """

    import matplotlib.pyplot as plt

    # (Optional) Sum importance by feature group
    if combo_impt:
        idx_cprp = [i for i, s in enumerate(feat_names) if "cprp" in s]
        idx_func = [i for i, s in enumerate(feat_names) if "fagg" in s]
        idx_func += [i for i, s in enumerate(feat_names) if "func" in s]
        idx_prod = [i for i, s in enumerate(feat_names) if "pagg" in s]
        idx_prod += [i for i, s in enumerate(feat_names) if "pgen" in s]
        idx_prod += [i for i, s in enumerate(feat_names) if "pgrp" in s]
        idx_WFms = [i for i, s in enumerate(feat_names) if "WFmeas" in s]
        idx_mtrx = [i for i, s in enumerate(feat_names) if "mtrx" in s]
        idx_mtrx.sort()
        idx_mtrx = idx_mtrx[:2]
        importances = np.asarray(
            [
                np.sum(importances[idx_cprp]),
                np.sum(importances[idx_func]),
                np.sum(importances[idx_prod]),
                np.sum(importances[idx_mtrx]),
                np.sum(importances[idx_WFms]),
            ]
        )
        # (Optional) Sum variance by feature group
        if np.all(variances != None):
            variances = np.asarray(
                [
                    np.sum(variances[idx_cprp]),
                    np.sum(variances[idx_func]),
                    np.sum(variances[idx_prod]),
                    np.sum(variances[idx_mtrx]),
                    np.sum(variances[idx_WFms]),
                ]
            )
        feat_names = [
            "chemProperties",
            "functionalUses",
            "productCategories",
            "productMatrix",
            "WFmeasured",
        ]

    indices = np.argsort(importances)

    # (Optional) Add error bars
    if np.all(variances != None):
        err_bars = np.sqrt(variances)  # standard deviation
        fig, ax = plt.subplots()
        plt.grid(True)
        ax.barh(
            range(len(indices)),
            importances[indices],
            xerr=err_bars[indices],
            capsize=3,
            align="center",
        )
    else:
        fig, ax = plt.subplots()
        ax.barh(range(len(indices)), importances[indices], align="center")

    # Add grid lines
    plt.grid(False)
    xax_max = np.round_(np.amax(importances), decimals=1)
    ax.set_xticks(np.arange(0, xax_max + 0.05, xax_max / 5))
    ax.xaxis.grid(color="silver")
    ax.set_axisbelow(True)

    # Label parts of plot
    ax.set_title("Feature Importance")
    ax.set_xlabel("Relative Importance")
    ax.set_yticks(np.arange(len(feat_names)))
    ax.set_yticklabels([feat_names[i] for i in indices])
    # Add importance value labels at the end of bars
    if variances is None:
        for rect in ax.patches:
            # Get X and Y placement of label from rect
            x_value = rect.get_width()
            y_value = rect.get_y() + rect.get_height() / 2
            # Use X value as label and format number with one decimal place
            label = "{:.2f}".format(x_value)
            # Create annotation
            plt.annotate(
                label,
                (x_value, y_value),  # Place label at end of the bar
                xytext=(4, 0),  # Horizontally shift label
                textcoords="offset points",  # Interpret `xytext` as offset
                va="center",
                ha="left",
            )

    fig = matplotlib.pyplot.gcf()
    if combo_impt:
        fig.set_size_inches(10, 6)
    else:
        fig.set_size_inches(10, 10)
    if np.all(save_fig_name != None):
        savepdf(fig, "feature_importance_%s" % save_fig_name)
    plt.show()

In [38]:
def plot_conf_matrix(cm, classes, normalize=True, cmap=matplotlib.cm.Blues):
    """Print and plot a confusion matrix
    
    Normalization can be applied by setting `normalize=True`.
    
    Adapted from:
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """

    import matplotlib.pyplot as plt
    import itertools

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        title = "Normalized Confusion Matrix"
    else:
        title = "Confusion Matrix"

    np.set_printoptions(precision=2)
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.gcf().subplots_adjust(bottom=0.2)
    plt.ylabel("True weight fraction")
    plt.xlabel("Predicted weight fraction")

References for implementing the augmentation functions: 
* https://stackoverflow.com/questions/34226400/find-the-index-of-the-k-smallest-values-of-a-numpy-array
* https://stackoverflow.com/questions/22117834/how-do-i-return-a-list-of-the-3-lowest-values-in-another-list
* http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_distances.html

In [39]:
import random as pyrandom
from numpy import random
from sklearn.metrics.pairwise import cosine_distances
from sklearn.preprocessing import MinMaxScaler

# TODO: These variables might cause problems if they're separate
# Define feature mask for data augmentation
feat_names = X_enm.columns
col_mask = ["cprp" not in name for name in feat_names]


# Functions for different data augmentation methods


def random_augment(k, X_source, y_source, random_state, X, y):
    """Randomly samples source data to pair with target data."""

    if k == 0:
        return X, y

    pyrandom.seed(random_state)
    np.random.seed(random_state)

    # Number of samples to select
    n_samples = k * len(X)
    # Obtain indices for randomly sampling source data
    idx_match = np.random.choice(len(X_source), n_samples)
    # Select matching rows from source data
    X_match = X_source.iloc[idx_match, :]
    y_match = y_source[idx_match]
    # Append sampled source data to target data
    X_aug = np.concatenate((X, X_match))
    y_aug = np.concatenate((y, y_match))
    assert (
        X_aug.shape[0] == y_aug.shape[0]
    ), f"X_aug.shape={X_aug.shape}, y_aug.shape={y_aug.shape}"

    return X_aug, y_aug


def unsupervised_augment(k, X_source, y_source, random_state, X, y):
    """
    Unsupervised data augmentation
    
    Match "k" most similar source data samples to target data samples 
    based on the smallest cosine distance between target and source data 
    samples (i.e., in an supervised fashion).
    """

    if k == 0:
        return X, y

    pyrandom.seed(random_state)
    np.random.seed(random_state)

    # Cosine distance matrix using feature mask
    cosdist_samples = cosine_distances(X_source * col_mask, X * col_mask)
    # Loop over distance matrix in search of k-smallest distances
    idx_match = []
    for col in cosdist_samples.T:
        # Find organics data indices of k-smallest distances
        matches = np.argpartition(col, k)[:k]
        idx_match.extend(matches)
    # Select matching rows from source data
    X_match = X_source.iloc[idx_match, :]
    y_match = y_source[idx_match]
    # Append sampled source data to target data
    X_aug = np.concatenate((X, X_match))
    y_aug = np.concatenate((y, y_match))

    return X_aug, y_aug


def supervised_augment(k, X_source, y_source, random_state, X, y):
    """
    Supervised data augmentation
    
    Match "k" most similar source data samples to target data samples 
    based on the smallest average of cosine distance between samples 
    and distance between WF labels (i.e., in an supervised fashion).
    """

    if k == 0:
        return X, y

    pyrandom.seed(random_state)
    np.random.seed(random_state)

    # Cosine distance matrix using feature mask
    cosdist_samples = cosine_distances(X_source * col_mask, X * col_mask)
    # For supervised matching augmentation, also consider WF labels
    # Turn 1D label arrays into 2D arrays
    y_2d = np.tile(y, (len(y_source), 1))
    y_source_2d = np.tile(y_source, (len(y), 1)).transpose()
    # Get normalized distance between ENM and organics labels
    scaler = MinMaxScaler()
    dist_y = scaler.fit_transform(np.abs(y_2d - y_source_2d).astype(float))
    # Weighted average distances of features and labels
    dist_matrix = (0.95 * cosdist_samples) + (0.05 * dist_y)
    # Loop over distance matrix in search of k-smallest distances
    idx_match = []
    for col in dist_matrix.T:
        # Find organics data indices of k-smallest distances
        matches = np.argpartition(col, k)[:k]
        idx_match.extend(matches)
    # Select matching rows from source data
    X_match = X_source.iloc[idx_match, :]
    y_match = y_source[idx_match]
    # Append sampled source data to target data
    X_aug = np.concatenate((X, X_match))
    y_aug = np.concatenate((y, y_match))

    return X_aug, y_aug

In [40]:
from sklearn.pipeline import Pipeline
import sys


class AugmentingPipeline(Pipeline):
    def __init__(
        self,
        steps,
        *,
        augmentation_type=None,
        augmentation_k=None,
        augmentation_X_source=None,
        augmentation_y_source=None,
        augmentation_random_state=None,
        **kwargs,
    ):
        self.augmentation_type = augmentation_type
        self.augmentation_k = augmentation_k
        self.augmentation_X_source = augmentation_X_source
        self.augmentation_y_source = augmentation_y_source
        self.augmentation_random_state = augmentation_random_state
        super().__init__(steps, **kwargs)

    def get_params(self, deep=True):
        params = super().get_params(deep=deep)
        return {
            "augmentation_type": self.augmentation_type,
            "augmentation_k": self.augmentation_k,
            "augmentation_X_source": self.augmentation_X_source,
            "augmentation_y_source": self.augmentation_y_source,
            "augmentation_random_state": self.augmentation_random_state,
            **params,
        }

    def set_params(self, **kwargs):
        if "augmentation_type" in kwargs:
            self.augmentation_type = kwargs["augmentation_type"]
        if "augmentation_k" in kwargs:
            self.augmentation_k = kwargs["augmentation_k"]
        if "augmentation_X_source" in kwargs:
            self.augmentation_X_source = kwargs["augmentation_X_source"]
        if "augmentation_y_source" in kwargs:
            self.augmentation_y_source = kwargs["augmentation_y_source"]
        super().set_params(**kwargs)
        return self

    def _augment(self, X, y):
        """Apply specified data augmentation function"""

        return self.augmentation_type(
            self.augmentation_k,
            self.augmentation_X_source,
            self.augmentation_y_source,
            self.augmentation_random_state,
            X,
            y,
        )

    def fit(self, X, y=None, **fit_params):
        # print("X dim", X.shape, file=sys.stderr)
        X, y = self._augment(X, y)  # Apply data augmentation
        # print("X_aug dim", X.shape, file=sys.stderr)
        # import pdb; pdb.set_trace()  # Option to run debugger
        return super().fit(X, y, **fit_params)

    def fit_transform(self, X, y=None, **fit_params):
        """fit_transform but with data augmentation. Only applies to training data."""
        print("fit_transform was called.", file=sys.stderr)
        X, y = self._augment(X, y)  # Apply data augmentation
        return super().fit_transform(X, y, **fit_params)

    def transform(self, X, y=None, **fit_params):
        print("transform was called.", file=sys.stderr)
        return super().transform(X, y, **fit_params)

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score


def apply_model_opt(
    classifiers,
    params,
    X_target=X_enm,
    y_target=bin_enm,
    cust_folds=None,
    random_state=None,
    n_jobs=3,
):

    """
    Optimize classifier parameters.
    
    Returns table of performance results across a grid of parameters.
    """

    # Set smallest class size as number of CV folds for leave-one-out CV
    _, class_counts = np.unique(y_target, return_counts=True)
    n_folds = min(class_counts)
    # Ignore n_folds above if custom CV fold is specified
    if cust_folds:
        n_folds = cust_folds
    else:
        n_folds = n_folds

    # Apply
    helper = EstimatorSelectionHelper(classifiers, params)
    helper.fit(
        X_target, y_target, n_jobs=n_jobs, cv=n_folds, scoring="balanced_accuracy",
    )
    results = helper.score_summary(sort_by="mean_score")
    results.columns = [col.split("__")[-1] for col in results.columns]
    # results["augmentation_type"] = [str(i).split(" ")[1] for i in results.loc[:, "augmentation_type"]]

    return results.infer_objects()

In [42]:
# @notify_on_complete # play sound at completion
def model_eval(
    classifier,
    augmentation_type,
    augmentation_k,
    X=X_enm,
    y=bin_enm,
    X_source=X_source,
    y_source=bin_source,
    random_state=np.arange(30),
    cust_folds=None,
    save_fig_name=None,
    show_feat_impt=True,
    show_conf_matrix=True,
):
    """Fit execute and evaluate a classifier using hyperparameters.
    
    1) Fit custom model pipeline that performs data augmentation and 
    normalization on training data using optimized parameters and 
    stratified k-fold cross validation;
    3) Execute optimized model and summarize its accuracy in a confusion 
    matrix broken down by WF bins. Formatted confusion matrices are saved as 
    .pdf files.
    
    Arguments
    ---------
    classifier : function
    augmentation_type : function
    k : int
        The number of organics samples to match with each ENM sample.
    X : DataFrame (default=X_enm)
        Target feature data.
    y : ndarray (default=bin_enm)
        Target WF bin data
    X_source : DataFrame (default=X_enm)
        Source feature data.
    y_source : ndarray (default=bin_enm)
        Source WF bin data.
    random_state : ndarray (default=np.arange(30))
        Option to set the seed for CV.
    save_fig_name : string (default=None)
        A unique string used at the end of confusion matrix and feature 
        importance (rfc-only) file names for exporting the figures as .pdf; 
        `None` indicates that no figures should be saved
    show_feat_impt : bool (default=False)
        Only applicable when classifier is 'rfc'; `True` takes results from 
        an rfc as input to plot a bar graph of feature importance.
    show_cnf_matrix : bool (default=False)
        `True` results in matrix graphics being printed as output
    """
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import StratifiedKFold
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import confusion_matrix, balanced_accuracy_score
    import matplotlib.pyplot as plt

    def standard_error(bal_accu):
        """Calculate standard error from an array of balanced accuracies."""
        n = bal_accu.size
        samp_var = np.var(bal_accu, ddof=1)
        return np.sqrt(samp_var / n)

    feat_names = X.columns.values
    X = np.array(X)
    n_b = len(np.unique(y))  # Check number of bins

    # Set smallest class size as number of CV folds for leave-one-out CV
    _, class_counts = np.unique(y, return_counts=True)
    n_folds = min(class_counts)
    # Custom folds override
    if cust_folds:
        n_folds = cust_folds
    else:
        n_folds = n_folds

    augmentation_kwargs = {
        "augmentation_X_source": X_source,
        "augmentation_y_source": y_source,
        "augmentation_type": augmentation_type,
        "augmentation_k": augmentation_k,
    }

    # Placeholders for confusion matrix (cm), feature importance
    cm_cum_state = np.zeros([n_b, n_b])
    arr_norm_state_avg = []
    std_err = []
    impt_cum = np.zeros(len(feat_names))

    for state in random_state:
        # CV settings
        skfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=state)
        # Placeholders
        arr_cm_norm_fold = np.zeros([n_folds, n_b, n_b])
        arr_norm_fold_avg = []
        impt_state = np.zeros(len(feat_names))

        # Fit and run pipeline
        for i, (train_index, test_index) in enumerate(skfold.split(X, y)):
            # Split data
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]
            # Pipeline with data augmentation
            pipe = AugmentingPipeline(
                [
                    ("scale", MinMaxScaler()),
                    ("estimator", classifier.set_params(random_state=state)),
                ],
                **augmentation_kwargs
            )
            pipe.fit(X_train, y_train)

            # Optional calculate feature importance (RFC only)
            if show_feat_impt:
                importances = classifier.feature_importances_
                impt_state += importances

            # Write prediction results to confusion matrix
            cm_fold = np.zeros([n_b, n_b])
            cm_fold = confusion_matrix(y_test, pipe.predict(X_test))  # size 3x3
            # Normalize balanced accuracy
            arr_cm_norm_fold[i, :, :] = (
                cm_fold.astype("float") / cm_fold.sum(axis=1)[:, np.newaxis]
            )  # 10x3x3
            # Get balanced average proportion of correct classifications
            arr_norm_fold_avg.append(
                arr_cm_norm_fold[i, :, :].diagonal().mean()
            )  # size 10

        # Accumulate data for each fold (n_fold total) over 30 trials
        cm_cum_state += np.average(arr_cm_norm_fold, axis=0)  # size 3x3
        arr_norm_state_avg.append(np.average(arr_norm_fold_avg))  # size 30
        std_err.append(standard_error(bal_accu=np.array(arr_norm_fold_avg)))  # size 30
        if show_feat_impt:
            impt_cum += impt_state

    # Average over 30 trials
    cm_avg_all = cm_cum_state / len(random_state)  # size 3x3
    std_err_avg = np.average(np.array(std_err))
    impt_avg = np.asarray(
        [i / (len(random_state) * n_folds) for i in impt_cum], dtype=np.float64,
    )
    # Average normalized balanced accuracy overall
    bal_accu_avg = cm_avg_all.diagonal().mean()

    # Plot and save normalized confusion matrix, optional feature importance
    fig = plt.figure()
    plot_conf_matrix(cm_avg_all, classes=["low", "mid", "high"])
    if np.all(save_fig_name != None):
        savepdf(fig, "confusion_norm_%s" % save_fig_name)
    if not show_conf_matrix:
        plt.close(fig)
    if show_feat_impt:
        plot_feat_impt(
            feat_names,
            impt_avg,
            variances=None,
            save_fig_name=save_fig_name,
            combo_impt=False,
        )

    # Set output based on chosen classifier
    if show_feat_impt:
        return arr_norm_state_avg, std_err, impt_avg
    else:
        return arr_norm_state_avg, std_err

In [43]:
def run_hyperparams(df, random_state, cust_folds=None, show_feat_impt=True):
    """
    Apply hyperparameters for model evaluation.
    
    Relies on `model_eval' function.
    """

    # Select for highest performing parameters (averaged across 10 folds)
    df["augmentation_type"] = [
        str(i).split(" ")[1] for i in df.loc[:, "augmentation_type"]
    ]
    feat_subset = ["estimator", "augmentation_k", "augmentation_type"]
    df = df.drop_duplicates(subset=feat_subset, keep="first")
    df = df.sort_values(by=feat_subset, ascending=[False, True, True])
    df.columns = [col.split("__")[-1] for col in df.columns]
    df = df.reset_index(drop=True)

    # Placeholder lists for output
    df[["score_avg", "score_list", "std_err", "impt_avg"]] = np.nan
    df[["score_avg", "score_list", "std_err", "impt_avg"]] = df[
        ["score_avg", "score_list", "std_err", "impt_avg"]
    ].astype(object)

    for row in df.index:
        # Dictionary of classifier parameters, dropping nans
        cls_kwargs = df.iloc[row, -7:-4].dropna().to_dict()
        # Classifier function using dict
        if df.loc[row, "estimator"] == "SVC":
            classifier = SVC(kernel="rbf", class_weight="balanced", **cls_kwargs)
        else:
            classifier = RandomForestClassifier(class_weight="balanced", **cls_kwargs)
        # Make sure augmentation_type is a function
        augmentation_type = df.augmentation_type[row]
        dispatcher = {
            "random_augment": random_augment,
            "unsupervised_augment": unsupervised_augment,
            "supervised_augment": supervised_augment,
        }
        if isinstance(augmentation_type, str):
            augmentation_type = dispatcher[augmentation_type]
        # Name figures using concatenated strings
        save_fig_name = "_".join(map(str, list(df.loc[row, feat_subset])))

        # Make parameter dict
        params = {
            "classifier": classifier,
            "augmentation_type": augmentation_type,
            "augmentation_k": df.augmentation_k[row],
            "random_state": random_state,
            "cust_folds": cust_folds,
            "save_fig_name": save_fig_name,
            "show_feat_impt": show_feat_impt,
        }
        if show_feat_impt:
            score, err, impt = model_eval(**params)
            df.score_list[row] = score
            df.std_err[row] = err
            df.impt_avg[row] = impt
        else:
            score, err = model_eval(**params)
            df.score_list[row] = score
            df.std_err[row] = err
        df.score_avg[row] = np.average(df.score_list[row])
    cols_report = [
        "estimator",
        "augmentation_type",
        "augmentation_k",
        "score_avg",
        "score_list",
        "std_err",
    ]
    if show_feat_impt:
        cols_report = [cols_report] + "impt_avg"

    return df[cols_report]

# Export

In [18]:
if __name__ == "__main__":
    !jupyter nbconvert --to script functions.ipynb

[NbConvertApp] Converting notebook functions.ipynb to script
[NbConvertApp] Writing 36369 bytes to functions.py
